In [35]:
from pyspark.sql import SparkSession
# import pandas as pd

spark = SparkSession.builder.appName("uber-date-trips-sql").getOrCreate()

directory = "/home/steve/Capstone/IIS/Stock/Tickers/old_data"
filename = "AAPL.csv"

data = spark.read.csv(f"file:///{directory}/{filename}", inferSchema = True, header = True)

# print(data)

In [38]:
data.head()

22/10/18 14:42:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Date, Open, High, Low, Close, Adj Close, Volume
 Schema: _c0, Date, Open, High, Low, Close, Adj Close, Volume
Expected: _c0 but found: 
CSV file: file:///home/steve/Capstone/IIS/Stock/Tickers/old_data/AAPL.csv


Row(_c0=0, Date=datetime.datetime(2012, 1, 3, 0, 0), Open=14.621429, High=14.732143, Low=14.607143, Close=14.686786, Adj Close=12.540046, Volume=302220800)

In [36]:
from pyspark import SparkConf, SparkContext
import pandas as pd

conf = SparkConf().setMaster("local").setAppName("uber-date-trips")
sc = SparkContext(conf=conf)

directory = "/home/steve/Capstone/IIS/Stock/Tickers/old_data"
filename = "GOOG.csv"

lines = sc.textFile(f"file:///{directory}/{filename}")

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [31]:
header = lines.first() 
filtered_lines = lines.filter(lambda row:row != header) 

AttributeError: 'NoneType' object has no attribute 'sc'

In [16]:
filtered_lines.show(5)

AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [19]:
data.show(5)

22/10/17 20:09:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Date, Open, High, Low, Close, Adj Close, Volume
 Schema: _c0, Date, Open, High, Low, Close, Adj Close, Volume
Expected: _c0 but found: 
CSV file: file:///home/steve/Capstone/IIS/Stock/AAPL.csv
+---+-------------------+---------+---------+---------+---------+---------+---------+
|_c0|               Date|     Open|     High|      Low|    Close|Adj Close|   Volume|
+---+-------------------+---------+---------+---------+---------+---------+---------+
|  0|2012-01-03 00:00:00|14.621429|14.732143|14.607143|14.686786|12.540045|302220800|
|  1|2012-01-04 00:00:00|14.642857|    14.81|14.617143|14.765714|12.607439|260022000|
|  2|2012-01-05 00:00:00|14.819643|14.948214|14.738214|14.929643|12.747405|271269600|
|  3|2012-01-06 00:00:00|14.991786|15.098214|14.972143|15.085714|12.880666|318292800|
|  4|2012-01-09 00:00:00|15.196429|15.276786|15.048214|15.061786|12.860234|394024400|
+---+-------------------

In [20]:
data.createOrReplaceTempView("APPLE_old_data")

In [21]:
spark.sql("select * from APPLE_old_data limit 5").show()

22/10/17 20:11:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Date, Open, High, Low, Close, Adj Close, Volume
 Schema: _c0, Date, Open, High, Low, Close, Adj Close, Volume
Expected: _c0 but found: 
CSV file: file:///home/steve/Capstone/IIS/Stock/AAPL.csv
+---+-------------------+---------+---------+---------+---------+---------+---------+
|_c0|               Date|     Open|     High|      Low|    Close|Adj Close|   Volume|
+---+-------------------+---------+---------+---------+---------+---------+---------+
|  0|2012-01-03 00:00:00|14.621429|14.732143|14.607143|14.686786|12.540045|302220800|
|  1|2012-01-04 00:00:00|14.642857|    14.81|14.617143|14.765714|12.607439|260022000|
|  2|2012-01-05 00:00:00|14.819643|14.948214|14.738214|14.929643|12.747405|271269600|
|  3|2012-01-06 00:00:00|14.991786|15.098214|14.972143|15.085714|12.880666|318292800|
|  4|2012-01-09 00:00:00|15.196429|15.276786|15.048214|15.061786|12.860234|394024400|
+---+-------------------

In [27]:
spark.sql("select Date, Open from APPLE_old_data").show()

+-------------------+---------+
|               Date|     Open|
+-------------------+---------+
|2012-01-03 00:00:00|14.621429|
|2012-01-04 00:00:00|14.642857|
|2012-01-05 00:00:00|14.819643|
|2012-01-06 00:00:00|14.991786|
|2012-01-09 00:00:00|15.196429|
|2012-01-10 00:00:00|15.211071|
|2012-01-11 00:00:00|15.095714|
|2012-01-12 00:00:00|15.081429|
|2012-01-13 00:00:00|14.989286|
|2012-01-17 00:00:00|    15.15|
|2012-01-18 00:00:00|15.248571|
|2012-01-19 00:00:00|  15.3625|
|2012-01-20 00:00:00|  15.2675|
|2012-01-23 00:00:00|15.095357|
|2012-01-24 00:00:00|15.182143|
|2012-01-25 00:00:00|    16.23|
|2012-01-26 00:00:00|16.012857|
|2012-01-27 00:00:00|15.869286|
|2012-01-30 00:00:00|15.918214|
|2012-01-31 00:00:00| 16.27107|
+-------------------+---------+
only showing top 20 rows



In [23]:
spark.sql("select Date, count(*) as open \
    from (select split(Date, ' ')[0] \
          as pickup_date from APPLE_old_data) \
          group by date ").explain(True)

AnalysisException: Column 'Date' does not exist. Did you mean one of the following? [__auto_generated_subquery_name.pickup_date]; line 1 pos 7;
'Aggregate ['date], ['Date, count(1) AS open#161L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(Date#51 as string),  , -1)[0] AS pickup_date#160]
      +- SubqueryAlias apple_old_data
         +- View (`APPLE_old_data`, [_c0#50,Date#51,Open#52,High#53,Low#54,Close#55,Adj Close#56,Volume#57])
            +- Relation [_c0#50,Date#51,Open#52,High#53,Low#54,Close#55,Adj Close#56,Volume#57] csv


In [34]:
sc.stop()